In [1]:
# Configure asyncio for Jupyter notebook
# This allows nested event loops which ib_insync needs in a notebook environment
import nest_asyncio
nest_asyncio.apply()

# Phase 4: Order Management Demo

This notebook demonstrates the order management functionality from Phase 4 of the IBKR Gateway project.

**Prerequisites:**
- IBKR Gateway or TWS running in paper trading mode
- Default connection: `127.0.0.1:4002`

## Features Demonstrated:
1. **Order Preview** - Estimate order impact without placing
2. **Safety Guards** - ORDERS_ENABLED protection
3. **Order Placement** - Place limit orders (paper mode)
4. **Order Status** - Track order state
5. **Order Cancellation** - Cancel open orders
6. **Order Registry** - In-memory tracking
7. **Error Handling** - Validation and exception handling

## Safety Notes:
- By default, `ORDERS_ENABLED=false` prevents real orders
- Set `ORDERS_ENABLED=true` in paper mode to test real placement
- **NEVER** enable orders in live mode without proper safety measures

In [2]:
# Add parent directory to path for imports
import sys
import os
sys.path.insert(0, '..')

from ibkr_core import IBKRClient
from ibkr_core.models import OrderSpec, SymbolSpec
from ibkr_core.config import get_config, reset_config

from ibkr_core.orders import (
    # Core functions
    preview_order,
    place_order,
    cancel_order,
    get_order_status,
    get_open_orders,
    # Registry
    get_order_registry,
    # Validation
    validate_order_spec,
    OrderError,
    OrderValidationError,
    OrderPlacementError,
    OrderNotFoundError,
    OrderPreviewError,
)

## 1. Environment Setup

Check the current configuration and safety settings.

In [3]:
# Reset and load config
reset_config()

# Check current settings
print("Current Environment Configuration")
print("=" * 40)
print(f"TRADING_MODE:   {os.environ.get('TRADING_MODE', 'paper (default)')}")
print(f"ORDERS_ENABLED: {os.environ.get('ORDERS_ENABLED', 'false (default)')}")

config = get_config()
print("\nActive Config:")
print(f"  trading_mode:   {config.trading_mode}")
print(f"  orders_enabled: {config.orders_enabled}")
print(f"  gateway_port:   {config.ibkr_gateway_port}")

Current Environment Configuration
TRADING_MODE:   paper
ORDERS_ENABLED: true

Active Config:
  trading_mode:   paper
  orders_enabled: True
  gateway_port:   4002


## 2. Connect to IBKR Gateway

In [4]:
# Create client and connect
client = IBKRClient(mode="paper")
client.connect(timeout=10)

print(f"Connected: {client.is_connected}")
print(f"Mode: {client.mode}")
print(f"Accounts: {client.managed_accounts}")

Connected: True
Mode: paper
Accounts: ['DUM096342']


Error 300, reqId 4: Can't find EId with tickerId:4
Error 300, reqId 7: Can't find EId with tickerId:7
Error 300, reqId 8: Can't find EId with tickerId:8


## 3. Order Preview

Preview orders to see estimated execution prices and notional values without actually placing them.

In [5]:
# Create an AAPL symbol spec
aapl_spec = SymbolSpec(
    symbol="AAPL",
    securityType="STK",
    exchange="SMART",
    currency="USD",
)

# Preview a market order
mkt_order = OrderSpec(
    instrument=aapl_spec,
    side="BUY",
    quantity=10,
    orderType="MKT",
    tif="DAY",
)

print("Preview: AAPL Market Order (BUY 10)")
print("=" * 50)
preview = preview_order(client, mkt_order)

print(f"Estimated Price:    ${preview.estimatedPrice:,.2f}")
print(f"Estimated Notional: ${preview.estimatedNotional:,.2f}")
print(f"Warnings:           {preview.warnings}")

Preview: AAPL Market Order (BUY 10)
Estimated Price:    $270.68
Estimated Notional: $2,706.80
Warnings:           ['Preview is simulated using current market data']


In [6]:
# Preview a limit order
lmt_order = OrderSpec(
    instrument=aapl_spec,
    side="SELL",
    quantity=5,
    orderType="LMT",
    limitPrice=350.00,  # Far from market
    tif="GTC",
)

print("\nPreview: AAPL Limit Order (SELL 5 @ $350)")
print("=" * 50)
preview = preview_order(client, lmt_order)

print(f"Estimated Price:    ${preview.estimatedPrice:,.2f}")
print(f"Estimated Notional: ${preview.estimatedNotional:,.2f}")


Preview: AAPL Limit Order (SELL 5 @ $350)
Estimated Price:    $350.00
Estimated Notional: $1,750.00


In [7]:
# Preview a MES futures order
mes_spec = SymbolSpec(
    symbol="MES",
    securityType="FUT",
    exchange="CME",
    currency="USD",
)

mes_order = OrderSpec(
    instrument=mes_spec,
    side="BUY",
    quantity=1,
    orderType="LMT",
    limitPrice=5000.00,
    tif="DAY",
)

print("\nPreview: MES Futures Order (BUY 1 @ $5000)")
print("=" * 50)
try:
    preview = preview_order(client, mes_order)
    print(f"Estimated Price:    ${preview.estimatedPrice:,.2f}")
    print(f"Estimated Notional: ${preview.estimatedNotional:,.2f}")
    print("(Note: MES has $5 multiplier)")
except OrderPreviewError as e:
    print(f"Preview failed: {e}")


Preview: MES Futures Order (BUY 1 @ $5000)
Estimated Price:    $5,000.00
Estimated Notional: $25,000.00
(Note: MES has $5 multiplier)


## 4. Safety Demonstration: ORDERS_ENABLED=false

When `ORDERS_ENABLED=false` (the default), orders are simulated and never sent to IBKR.

In [8]:
# Ensure ORDERS_ENABLED=false
os.environ["ORDERS_ENABLED"] = "false"
reset_config()

print("Safety Mode: ORDERS_ENABLED=false")
print("=" * 50)
print(f"orders_enabled: {get_config().orders_enabled}")

# Try to place an order
order = OrderSpec(
    instrument=aapl_spec,
    side="BUY",
    quantity=100,
    orderType="MKT",
    tif="DAY",
)

result = place_order(client, order)

print("\nOrder Result:")
print(f"  Status:   {result.status}")
print(f"  Order ID: {result.orderId}")
print(f"  Errors:   {result.errors}")

if result.status == "SIMULATED":
    print("\n[SAFE] Order was simulated - no real order sent to IBKR!")

Safety Mode: ORDERS_ENABLED=false
orders_enabled: False

Order Result:
  Status:   SIMULATED
  Order ID: None
  Errors:   ['Order not placed: ORDERS_ENABLED=false (simulation mode)']

[SAFE] Order was simulated - no real order sent to IBKR!


## 5. Validation Errors

See how invalid orders are caught before being sent.

In [9]:
print("Validation Error Examples")
print("=" * 50)

# Example 1: Market order with limit price (invalid)
try:
    invalid_order = OrderSpec(
        instrument=aapl_spec,
        side="BUY",
        quantity=1,
        orderType="MKT",
        limitPrice=150.00,  # Invalid for MKT!
        tif="DAY",
    )
    errors = validate_order_spec(invalid_order)
    print("\n1. MKT order with limit price:")
    print(f"   Errors: {errors}")
except Exception as e:
    print(f"\n1. Caught Pydantic validation: {e}")

# Example 2: Limit order without price
try:
    invalid_order = OrderSpec(
        instrument=aapl_spec,
        side="BUY",
        quantity=1,
        orderType="LMT",
        limitPrice=None,  # Required for LMT!
        tif="DAY",
    )
    errors = validate_order_spec(invalid_order)
    print("\n2. LMT order without limit price:")
    print(f"   Errors: {errors}")
except Exception as e:
    print(f"\n2. Caught validation: {e}")

# Example 3: Negative quantity
try:
    from pydantic import ValidationError
    invalid_order = OrderSpec(
        instrument=aapl_spec,
        side="BUY",
        quantity=-1,  # Invalid!
        orderType="MKT",
        tif="DAY",
    )
except ValidationError:
    print("\n3. Negative quantity:")
    print("   Pydantic error: quantity must be positive")

Validation Error Examples

1. MKT order with limit price:
   Errors: ['Market orders (MKT) cannot have a limit price']

2. LMT order without limit price:
   Errors: ['Limit orders (LMT) require a limit price']

3. Negative quantity:
   Pydantic error: quantity must be positive


## 6. Real Order Placement (Paper Mode)

**IMPORTANT:** The following cells demonstrate real order placement. 
- Ensure you're in **PAPER** trading mode
- Set `ORDERS_ENABLED=true` to enable
- Orders will use far-from-market prices to avoid fills

To enable real orders, uncomment and run the cell below:

In [10]:
# UNCOMMENT BELOW TO ENABLE REAL ORDER PLACEMENT (PAPER ONLY!)
os.environ["ORDERS_ENABLED"] = "true"
reset_config()
print(f"ORDERS_ENABLED: {get_config().orders_enabled}")
print("WARNING: Real orders will be sent to IBKR paper account!")

ORDERS_ENABLED: True


In [11]:
# Place a far-from-market limit order (won't fill)
print("Placing Far-From-Market Limit Order")
print("=" * 50)

test_order = OrderSpec(
    instrument=aapl_spec,
    side="BUY",
    quantity=1,
    orderType="LMT",
    limitPrice=1.00,  # Very low - won't fill
    tif="DAY",
)

result = place_order(client, test_order)

print(f"Status:   {result.status}")
print(f"Order ID: {result.orderId}")

if result.status == "ACCEPTED":
    print("\nOrder Status:")
    print(f"  IBKR Status: {result.orderStatus.status}")
    print(f"  Filled:      {result.orderStatus.filledQuantity}")
    print(f"  Remaining:   {result.orderStatus.remainingQuantity}")
elif result.status == "SIMULATED":
    print(f"\n[SIMULATED] {result.errors[0]}")
    
# Save order_id for later
test_order_id = result.orderId

Placing Far-From-Market Limit Order
Status:   ACCEPTED
Order ID: 9

Order Status:
  IBKR Status: SUBMITTED
  Filled:      0.0
  Remaining:   1.0


In [12]:
# Check order status
if test_order_id:
    print(f"Checking Status for Order: {test_order_id}")
    print("=" * 50)
    
    try:
        status = get_order_status(client, test_order_id)
        print(f"Order ID:     {status.orderId}")
        print(f"Status:       {status.status}")
        print(f"Filled:       {status.filledQuantity}")
        print(f"Remaining:    {status.remainingQuantity}")
        print(f"Avg Price:    ${status.avgFillPrice:.2f}")
        print(f"Last Update:  {status.lastUpdate}")
    except OrderNotFoundError:
        print("Order not found (may be simulated mode)")
else:
    print("No order to check (simulated mode)")

Checking Status for Order: 9
Order ID:     9
Status:       SUBMITTED
Filled:       0.0
Remaining:    1.0
Avg Price:    $0.00
Last Update:  2025-12-19 18:32:37.743244+00:00


In [13]:
# Cancel the order
if test_order_id:
    print(f"Cancelling Order: {test_order_id}")
    print("=" * 50)
    
    cancel_result = cancel_order(client, test_order_id)
    print(f"Status:  {cancel_result.status}")
    print(f"Message: {cancel_result.message}")
else:
    print("No order to cancel (simulated mode)")

Cancelling Order: 9
Status:  CANCELLED
Message: Order cancelled (IBKR status: Cancelled)


## 7. Order Registry

The in-memory order registry tracks all orders placed during this session.

In [14]:
registry = get_order_registry()

print("Order Registry")
print("=" * 50)
print(f"Total orders tracked: {registry.size}")

all_orders = registry.all_orders()
if all_orders:
    print("\nRegistered Orders:")
    for order in all_orders:
        print(f"  {order['order_id']}: {order['symbol']} {order['side']} {order['quantity']}")
else:
    print("\nNo orders registered (orders may be in simulated mode)")

Order Registry
Total orders tracked: 1

Registered Orders:
  9: AAPL BUY 1.0


## 8. Open Orders Query

In [15]:
print("Open Orders")
print("=" * 50)

open_orders = get_open_orders(client)

if open_orders:
    print(f"{'Order ID':<15} {'Symbol':<10} {'Side':<6} {'Qty':>6} {'Type':<6} {'Status':<15}")
    print("-" * 65)
    for order in open_orders:
        print(f"{order['order_id']:<15} {order['symbol']:<10} {order['side']:<6} {order['quantity']:>6} {order['order_type']:<6} {order['status']:<15}")
else:
    print("No open orders")

Open Orders
No open orders


## 9. Order Types Summary

In [16]:
print("Supported Order Types")
print("=" * 50)
print("""
MKT (Market):
  - Executes immediately at current market price
  - No price parameters needed
  
LMT (Limit):
  - Executes at limit price or better
  - Requires: limitPrice > 0
  
STP (Stop):
  - Triggers market order when stop price reached
  - Requires: stopPrice > 0
  
STP_LMT (Stop-Limit):
  - Triggers limit order when stop price reached
  - Requires: stopPrice > 0, limitPrice > 0

Time-In-Force (TIF):
  - DAY: Valid until end of trading day
  - GTC: Good-til-cancelled (remains until filled/cancelled)
  - IOC: Immediate-or-cancel (fill what you can, cancel rest)
  - FOK: Fill-or-kill (complete fill or nothing)
""")

Supported Order Types

MKT (Market):
  - Executes immediately at current market price
  - No price parameters needed

LMT (Limit):
  - Executes at limit price or better
  - Requires: limitPrice > 0

STP (Stop):
  - Triggers market order when stop price reached
  - Requires: stopPrice > 0

STP_LMT (Stop-Limit):
  - Triggers limit order when stop price reached
  - Requires: stopPrice > 0, limitPrice > 0

Time-In-Force (TIF):
  - DAY: Valid until end of trading day
  - GTC: Good-til-cancelled (remains until filled/cancelled)
  - IOC: Immediate-or-cancel (fill what you can, cancel rest)
  - FOK: Fill-or-kill (complete fill or nothing)



## 10. Exception Hierarchy

In [17]:
print("Exception Hierarchy")
print("=" * 50)
print("""
OrderError (base)
  ├── OrderValidationError  - Invalid order parameters
  ├── OrderPlacementError   - Failed to place order
  ├── OrderCancelError      - Failed to cancel order
  ├── OrderNotFoundError    - Order not found in registry/IBKR
  └── OrderPreviewError     - Preview failed

config.TradingDisabledError - Used for safety checks
""")

# Verify inheritance
print("Inheritance verification:")
print(f"  OrderValidationError inherits OrderError: {issubclass(OrderValidationError, OrderError)}")
print(f"  OrderPlacementError inherits OrderError: {issubclass(OrderPlacementError, OrderError)}")
print(f"  OrderNotFoundError inherits OrderError: {issubclass(OrderNotFoundError, OrderError)}")

Exception Hierarchy

OrderError (base)
  ├── OrderValidationError  - Invalid order parameters
  ├── OrderPlacementError   - Failed to place order
  ├── OrderCancelError      - Failed to cancel order
  ├── OrderNotFoundError    - Order not found in registry/IBKR
  └── OrderPreviewError     - Preview failed

config.TradingDisabledError - Used for safety checks

Inheritance verification:
  OrderValidationError inherits OrderError: True
  OrderPlacementError inherits OrderError: True
  OrderNotFoundError inherits OrderError: True


## 11. Cleanup & Disconnect

In [18]:
# Cancel any remaining open orders
print("Cleaning up open orders...")
for trade in client.ib.openTrades():
    try:
        client.ib.cancelOrder(trade.order)
        print(f"  Cancelled: {trade.contract.symbol}")
    except Exception as e:
        print(f"  Error cancelling: {e}")

print("Done.")

Cleaning up open orders...
Done.


In [19]:
# Disconnect from gateway
client.disconnect()
print(f"Disconnected. Is connected: {client.is_connected}")

Disconnected. Is connected: False


## Summary

Phase 4 provides comprehensive order management with safety-first design.

### Core Functions:
- `preview_order(client, order_spec)` - Estimate order without placing
- `place_order(client, order_spec)` - Place order with safety checks
- `cancel_order(client, order_id)` - Cancel an order
- `get_order_status(client, order_id)` - Get current order status
- `get_open_orders(client)` - List all open orders

### Safety Features:
- `ORDERS_ENABLED=false` prevents all real orders (default)
- `TRADING_MODE=live` requires explicit override file
- Validation catches errors before IBKR call
- Optional notional/quantity guards

### Data Models:
- `OrderSpec` - Order specification (what to trade)
- `OrderPreview` - Estimated impact before placement
- `OrderResult` - Result of place_order call
- `OrderStatus` - Current order state
- `CancelResult` - Result of cancel_order call

### Key Features:
- In-memory order registry for session tracking
- Support for MKT, LMT, STP, STP_LMT order types
- Support for stocks and futures
- Comprehensive error handling
- Logging of all order operations